# Regresja logistyczna

W zeszły tygodniu poznaliśmy pierwszy klasyfikator $k$ najbliższych sąsiadów. Dziś będziemy kontynuować temat klasyfikacji. Zaczniemy od szczególnego przypadku jakim jest klasyfikacja binarna.

## Klasyfikacja binarna

Klasyfikacja binarna jest szczególnych rodzajem, ponieważ mamy tylko dwie klasy. Bardzo często zdarza się, że chcemy przewidzieć, czy zaszło dane zjawisko, np. czy będzie dziś padać deszcz, czy przydzielić kredyt hipoteczny, przeżycie katastrofy Titanica itd. Będziemy przyjmonować następującą konwencję:
 * klasa 1 - zaszło dane zjawisko (np. przeżył)
 * klasa 0 - nie zaszło dane zjawisko (np. umarł).

Klasyfikator może popełnić dwa rodzaje błędów:
 * przewidzieć 1, gdy w rzeczywistości było 0 (błąd I rodzaju, *false positive*, FP)
 * przewidzieć 0, gdy w rzeczywistości było 1 (błąd II rodzaju, *false negative*,FN).
 
Niestety te dwa błędy wykluczają się: jeżeli nasz model będzie zawsze zwracał klasę 0, to nigdy nie popełni błędu pierwszego rodzaju. Podobnie w sytuacji, gdy zawsze będzie zwracał wartość 1: nigdy nie popełni błędu drugiego rodzaju.

### Precyzja i pokrycie (ang. precision and recall)

Istnieją dwie miary, które mierzą precyzję i pokrycie:
 * **precyzja**: mierzy pewność klasyfikatora, czyli. błąd I rodzaju: $$ PRECISION = \frac{TP}{TP+FP} $$
 * **pokrycie**: mierzy zasięg klasyfikatora, czyli błąd II rodzaju: $$ RECALL = \frac{TP}{TP + FN} $$
 
Gdzie TP (*true positive*) to liczba elementów dobrze zaklasyfikowanych w klasie 1.

**przykład 1** Weźmy pod uwagę zestaw danych z Titanica (``dev-0``). Nasz model działa tylko na podstawie płci, tzn. przewiduje 1 (przeżycie) wyłącznie gdy zadana osoba jest kobietą. Przedstawmy wyniki w postaci tabeli (ang. confusion matrix):

|   | 1  | 0  |
|---|----|----|
| 1 | 36 | 10 |
| 0 | 13 | 75 |

Kolumny: stan faktyczny, wiersze: predykcja modelu.

Zatem: $$PRECISION = \frac{36}{36 + 10} = \frac{36}{46}$$ i $$ RECALL = \frac{36}{36 + 13} = \frac{36}{49} $$  

**zad. 1** Napisz fukcję, która oblicza precyzję i recall na podstawie tabeli. Dane będą podawane wierszami, tj. dla powyższego przykładu: ``[[36, 10], [13, 75]]``.

In [1]:
data = [[36, 10], [13, 75]]

def precision_and_recall(dane):
    pass

print(precision_and_recall(data))

None


**zad. 2** Zamiana klas. Oblicz precyzję i recall dla zadania, w którym chcemy przewidzieć, czy ktoś **nie** przeżył katastrofy Titanica. Wykorzystaj dane z przykładu 1.

**zad. 3** Wykorzystaj model z ostatniego zadania domowego i oblicz precyzję i pokrycie na zbiorze ``dev-0``. Wykorzystaj fukcję ``classification_report`` z modułu ``sklearn.metrics``.

## Regresja logistyczna

Wykorzystanie regresji logistyczne z pakietu ``sklearn`` nie różni się od wykorzystania innych modeli.

Na podstawie ogłoszeń mieszkań będziemy chcieli przewidzieć, czy dane mieszkanie znajduje się w centrum.

In [4]:
import pandas as pd

data = pd.read_csv("gratkapl-centrenrm.csv")
X = data[['Expected','Rooms','SqrMeters', 'Floor']]
Y = data['Centre']

In [5]:
from sklearn.model_selection import train_test_split

(train_X, test_X, train_Y, test_Y) = train_test_split(X, Y, test_size=0.2, random_state=42)

### Normalizacja danych

Wykonajmy normalizacje danych wejściowych (odjęcie średniej i podzielenie przez otchylenie std). Wykorzystamy do tego narzędzie ``StandardScaler``.

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

scaler.fit(train_X)

train_X = scaler.transform(train_X)

**zad. 4** Korzystając z dokumentacji wyświetl średnią dla każdej z kolumn wejściowych.

Wytrenujmy model regresji logistycznej:

In [10]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

model.fit(train_X, train_Y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

**zad. 5** Wyświetl parametry modelu.

In [13]:
predicted = model.predict(scaler.transform(test_X)) # dane testowe też muszą zostać znormalizawane.

In [14]:
from sklearn.metrics import classification_report

In [15]:
print(classification_report(test_Y, predicted))

             precision    recall  f1-score   support

          0       0.73      0.97      0.83       288
          1       0.58      0.09      0.16       116

avg / total       0.68      0.72      0.64       404



### Mierzenie zależności pomiędzy precyzją i pokryciem

W najlepszym przypadku chcielibyśmy zminimalizować błędu obu rodzajów. Niestety, nie zawsze jest to możliwe. Bardzo często można zwiększyć pokrycie kosztem precyzji i na odwrót: zwiększyć precyzję kosztem pokrycia.

Dlatego wprowadza się dwie miary:
 * F-score: $$ FSCORE = \frac{2}{\frac{1}{PRECISON} + \frac{1}{RECALL}} $$
 * Krzywa [ROC](https://en.wikipedia.org/wiki/Receiver_operating_characteristic)

**zad. (domowe)** Uruchom regresję logistyczną na zbiorze Titanic:
 * Wykorzystaj arkusz z poprzedniego zadania domowego.
 * Wykonaj normalizacje danych na wybranych kolumnach? Jak zmieniły się wyniki?
 * korzystając z implementacji z ``sklearn`` wyświetl krzywą ROC na ``dev-0``. 